In [1]:
import numpy as np
import pandas as pd

In [2]:
raw_data = pd.read_csv("dataset2.csv")

In [3]:
raw_data.head()

,URL,Target,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,https://locking-app-adverds.000webhostapp.com/...,yes,NaN,NaN,NaN,NaN
1,http://www.myhealthcarepharmacy.ca/wp-includes...,yes,NaN,NaN,NaN,NaN
2,http://code.google.com/p/pylevenshtein/,no,NaN,NaN,NaN,NaN
3,http://linkedin.com/,no,NaN,NaN,NaN,NaN
4,http://imageshack.com/f/219/cadir2yr3.jpg,no,NaN,NaN,NaN,NaN


In [4]:
raw_data['URL'].str.split("://").head()

0    [https, locking-app-adverds.000webhostapp.com/...
1    [http, www.myhealthcarepharmacy.ca/wp-includes...
2             [http, code.google.com/p/pylevenshtein/]
3                                [http, linkedin.com/]
4           [http, imageshack.com/f/219/cadir2yr3.jpg]
Name: URL, dtype: object

In [5]:
seperation_of_protocol = raw_data['URL'].str.split("://",expand = True)

In [6]:
seperation_of_protocol.head()

,0,1,2,3,4,5,6
0,https,locking-app-adverds.000webhostapp.com/payment-...,None,None,None,None,None
1,http,www.myhealthcarepharmacy.ca/wp-includes/js/jqu...,None,None,None,None,None
2,http,code.google.com/p/pylevenshtein/,None,None,None,None,None
3,http,linkedin.com/,None,None,None,None,None
4,http,imageshack.com/f/219/cadir2yr3.jpg,None,None,None,None,None


In [7]:
type(seperation_of_protocol)

pandas.core.frame.DataFrame

In [8]:
seperation_domain_name = seperation_of_protocol[1].str.split("/",1,expand = True)

C:\Users\dk382\AppData\Local\Temp\ipykernel_20036\2695327230.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  seperation_domain_name = seperation_of_protocol[1].str.split("/",1,expand = True)


In [9]:
seperation_domain_name.columns=["domain_name","address"]

In [10]:
splitted_data = pd.concat([seperation_of_protocol[0],seperation_domain_name],axis=1)

In [11]:
splitted_data.columns = ['protocol','domain_name','address']

In [12]:
splitted_data['is_phished'] = pd.Series(raw_data['Target'], index=splitted_data.index)

In [13]:
splitted_data

,protocol,domain_name,address,is_phished
0,https,locking-app-adverds.000webhostapp.com,payment-update-0.html?fb_source=bookmark_apps&...,yes
1,http,www.myhealthcarepharmacy.ca,wp-includes/js/jquery/ini.php,yes
2,http,code.google.com,p/pylevenshtein/,no
3,http,linkedin.com,,no
4,http,imageshack.com,f/219/cadir2yr3.jpg,no
...,...,...,...,...
1776,https,docs.google.com,document/u/1/,no
1777,http,www.charlestodd.com,wp-includes/pomo/adobe2.html,yes
1778,http,tslimpact.com,medsynaptic/wp-content/themes/twentyseventeen/...,yes
1779,http,daoudilorin11.mystagingwebsite.com,wp-content/plugins/ubh/acc/dir/68e1c/dir/car.php,yes


In [14]:
def long_url(l):
    l= str(l)
    """This function is defined in order to differntiate website based on the length of the URL"""
    if len(l) < 54:
        return 0
    elif len(l) >= 54 and len(l) <= 75:
        return 2
    return 1

In [15]:
splitted_data['long_url'] = raw_data['URL'].apply(long_url) 

In [16]:
splitted_data[splitted_data.long_url == 0] 

,protocol,domain_name,address,is_phished,long_url
2,http,code.google.com,p/pylevenshtein/,no,0
3,http,linkedin.com,,no,0
4,http,imageshack.com,f/219/cadir2yr3.jpg,no,0
6,http,www.7-zip.org,download.html,no,0
7,http,ebay.com,,no,0
...,...,...,...,...,...
1766,http,malomolk.com,nab/cardinfo.html,yes,0
1774,http,www.dpincsupport.com,,no,0
1775,https,bitcoin.org,en/,no,0
1776,https,docs.google.com,document/u/1/,no,0


In [17]:
def have_at_symbol(l):
    """This function is used to check whether the URL contains @ symbol or not"""
    if "@" in str(l):
        return 1
    return 0

In [18]:
splitted_data['having_@_symbol'] = raw_data['URL'].apply(have_at_symbol)

In [19]:
def redirection(l):
    """If the url has symbol(//) after protocol then such URL is to be classified as phishing """
    if "//" in str(l):
        return 1
    return 0

In [20]:
splitted_data['redirection_//_symbol'] = seperation_of_protocol[1].apply(redirection)

In [21]:
def prefix_suffix_seperation(l):
    if '-' in str(l):
        return 1
    return 0

In [22]:
splitted_data['prefix_suffix_seperation'] = seperation_domain_name['domain_name'].apply(prefix_suffix_seperation)

In [23]:
def sub_domains(l):
    l= str(l)
    if l.count('.') < 3:
        return 0
    elif l.count('.') == 3:
        return 2
    return 1

In [24]:
splitted_data['sub_domains'] = splitted_data['domain_name'].apply(sub_domains)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [26]:
x = splitted_data.columns[4:9]
x 

Index(['long_url', 'having_@_symbol', 'redirection_//_symbol',
       'prefix_suffix_seperation', 'sub_domains'],
      dtype='object')

In [28]:
y = pd.factorize(splitted_data['is_phished'])[0]
y 

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [29]:
clf = RandomForestClassifier(n_estimators=100,n_jobs=2,random_state=0)

clf.fit(splitted_data[x], y)

RandomForestClassifier(n_jobs=2, random_state=0)

In [30]:
test_data = pd.read_csv("dataset3.csv") 

In [31]:
clf.predict(test_data[x])

array([0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,

In [32]:
clf.predict_proba(test_data[x])[0:10]

array([[0.        , 0.88800564, 0.11199436],
       [0.        , 0.50510239, 0.49489761],
       [0.00135046, 0.32367919, 0.67497035],
       [0.        , 0.50510239, 0.49489761],
       [0.00135046, 0.32367919, 0.67497035],
       [0.00135046, 0.32367919, 0.67497035],
       [0.        , 0.53582536, 0.46417464],
       [0.00135046, 0.32367919, 0.67497035],
       [0.00135046, 0.32367919, 0.67497035],
       [0.00135046, 0.32367919, 0.67497035]])

In [33]:
preds = test_data.is_phished[clf.predict(test_data[x])]

In [34]:
actual = pd.Series(test_data['is_phished'])

In [35]:
confusion_matrix(actual,preds)

array([[ 94,  80],
       [ 64, 152]], dtype=int64)

In [37]:
accuracy_score(actual,preds)

0.6307692307692307

In [38]:
list(zip(splitted_data[x], clf.feature_importances_))

[('long_url', 0.35581616461435084),
 ('having_@_symbol', 0.012201419675347217),
 ('redirection_//_symbol', 0.005280064350323051),
 ('prefix_suffix_seperation', 0.5700748206742969),
 ('sub_domains', 0.05662753068568203)]